In [ ]:
# General setup for all examples
import foolbox.ext.native as fbn
import torchvision.models as models
import torch
import numpy as np

# instantiate a model
model = models.resnet18(pretrained=True).eval()
preprocessing = dict(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], axis=-3)
fmodel = fbn.models.PyTorchModel(model, bounds=(0, 1), preprocessing=preprocessing)

# get data and test the model
images, labels = fbn.utils.samples(fmodel, dataset='imagenet', batchsize=4)
print(fbn.utils.accuracy(fmodel, images, labels))
# -> 1.0

# for the rest of the tutorial, images are assumed to be properly normalised
mean = torch.from_numpy(np.array(preprocessing['mean'], dtype=np.float32))
std = torch.from_numpy(np.array(preprocessing['std'], dtype=np.float32))

mean = mean[None, :, None, None].to(images.device)
std = std[None, :, None, None].to(images.device)

images = (images - mean) / std

### Minimum example to use Brendel & Bethge attack

In [ ]:
# convert Foolbox attack to CleverHans attack
from cleverfool import convert_foolbox_attack
linf_brendel_bethge = convert_foolbox_attack(fbn.attacks.LinfinityBrendelBethgeAttack)

# run attack
x_fool = linf_brendel_bethge(model, images, labels=labels, clip_min=-10, clip_max=10)
y_pred_fool = model(x_fool).argmax(1)
print(float((y_pred_fool == labels).float().mean()))

### Using Brendel & Bethge with starting points selected from data set (to ensure valid starting points)

In [ ]:
# get all data
images, labels = fbn.utils.samples(fmodel, dataset='imagenet', batchsize=20)

# split images and labels into 'batches' (just to mirror the batches of standard large-scale datasets)
batches = [(images[:10], labels[:10]), (images[10:], labels[10:])]

# create attack that picks adversarials from given dataset of samples
from cleverfool import convert_cleverhans_model
fmodel = convert_cleverhans_model(model, images, clip_min=-10, clip_max=10)
init_attack = fbn.attacks.DatasetAttack(fmodel)

init_attack.feed(batches[0][0])   # feed 1st batch of inputs
init_attack.feed(batches[1][0])   # feed 2nd batch of inputs

# apply the Brendel & Bethge attack
x_fool = linf_brendel_bethge(model, images, labels=labels, clip_min=-10, clip_max=10, init_attack=init_attack)
y_pred_fool = model(x_fool).argmax(1)
print(float((y_pred_fool == labels).float().mean()))
# -> 0.0